In [2]:
!pip install transformers

     |████████████████████████████████| 2.5MB 7.2MB/s 
     |████████████████████████████████| 3.3MB 22.0MB/s 
     |████████████████████████████████| 901kB 40.8MB/s 


In [3]:
import torch

from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [5]:
question = "How many parameters does BERT-large have?"
answer_text = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."

In [52]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_ids = tokenizer.encode(question, answer_text, max_length=5000, truncation = True)

print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 68 tokens.


In [53]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [35]:
# Run our example through the model.
start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids]), return_dict=False) # The segment IDs to differentiate question from answer_text


In [36]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)

In [37]:
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Start with the first token.
answer = tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
    
    # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
    
    # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]

print('Answer: "' + answer + '"')

Answer: "bert"


In [83]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text, padding=True, truncation=True,max_length=1000, add_special_tokens = True)                   # ADD TRUNCATION & MAX_LENGTH !!!
    # OLD: input_ids = tokenizer.encode(question, answer_text, truncation = True, max_length=1000) 

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids]), return_dict=False) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    answer = full_sentence (tokens, answer_start, answer_end)
    

    print('Answer: "' + answer + '"')

In [84]:
def start_of_sentence (tokens, answer_start, answer_end):

    # Start with the first token and/or first letter in sentence
    if tokens[answer_start-1] == '.' or tokens[answer_start] == None:
        answer = tokens[answer_start]
    
    # Otherwise keep searching for the beginning of the sentence
    else: 
        while tokens[answer_start].isalnum() == False:                                    # ADD OR StATEMENT TOO??
            print(tokens[answer_start])
            answer_start-=1
    
    # Find the last letter in the sentence
    if tokens[answer_end].isalnum() == False:                                              # ADD OR StATEMENT TOO??
        while tokens[answer_end] != '.' or answer_end+1 >= len(tokens):
            answer_end+=1

    return tokens[answer_start], answer_start, answer_end

def update_tokens(answer, tokens, answer_start, answer_end):

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # If there is a punctuation mark, then combine without space
        elif tokens[i].isalnum() == False:
            answer += tokens[i]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    
    return answer

def update_punctuation (answer):
    
    # Set beginning of sentence to a letter
    while answer[0].isalnum() == False:
        answer = answer[1:]

    # Set end of sentence to a letter
    len_answer = len(answer)
    while answer[len_answer-1].isalnum() == False:
        answer = answer[:len_answer-1]
        len_answer-=1
    
    return answer

def full_sentence (tokens, answer_start, answer_end):

    # Update indices to include full sentence
    new_res = start_of_sentence(tokens, answer_start, answer_end)
    answer = new_res[0]
    answer_start = new_res[1]
    answer_end = new_res[2]

    # Update answer to include correct spacing with punctuation and combine ## tokens
    answer = update_tokens(answer, tokens, answer_start, answer_end)

    # Remove extraneous punctuation from start and end
    answer = update_punctuation(answer)

    # Capitalize the first letter and add period punctuation to end
    return answer[0].upper()+answer[1:]+'.'


In [85]:
# import textwrap

# # Wrap text to 80 characters.
# wrapper = textwrap.TextWrapper(width=80) 

bert_abstract = "Purchased Content. When you purchase an item of content, your content will be stored in a digital locker and you may view it an unlimited number of times for during your Locker Period. The “Locker Period” will be for at least 5 years from the date of your purchase (subject to the restrictions described in the YouTube Paid Service Terms of Service). Each item of purchased content may have a different Locker Period and you agree to the Locker Period before you order it. Pausing, stopping, or rewinding purchased content will not extend the Locker Period.  As noted in the YouTube Paid Service Terms of Service, if an item of purchased content becomes unavailable during the five year period from the purchase date, you may request a refund. For purchased content: you may view one stream of each item at a time, you may view up to 3 streams of different items at a time, you may authorize up to 5 devices for offline playback of Locker Video Content at a time and to authorize additional devices, you must deauthorize one of those 5 devices, you may only authorize the same device three times in any 12 month period and de-authorize the same device twice in any 12 month period, you may only deauthorize a total of 2 devices for offline playback every 90 days, and you may only authorize 3 Google accounts on the same device. Stream and offline playback limitations for purchased content apply regardless of which Google product (e.g., Google Play Movies & TV or YouTube) you access the content from."
#"YouTube Terms of Service Paid Service Terms of Service Paid Service Usage Rules Collecting Society Notices Copyright Notices Community Guidelines YouTube Paid Service Usage Rules As described in the YouTube Paid Service Terms of Service, how you access or use different types of Paid Services on YouTube may vary. The usage rules applicable to each type of Paid Service are detailed below, and may change over time as we add new features, devices and content to our service. You may also visit the YouTube Help Center for more information. Movies and Shows Rentals and Purchases General. Certain types of Paid Services allow you to purchase or rent individual Movies or TV shows made available by third parties. You may view rentals or purchases when online, with a broadband connection and logged onto your YouTube or Google account. You may also view certain rentals and purchases offline by downloading and viewing from a previously authorized device. Authorizing and deauthorizing a device requires you to be online.  Due to restrictions from our content providers, purchased or rented content may not be available for purchase in all countries or outside of your home country. Access on other Google Products. In some cases,  you may also receive access to your rentals and purchases through another Google product (e.g. Google Play). Your use of your rentals and purchases from YouTube shall be governed by the YouTube usage rules, but these usage rules will govern your use on these Google products in addition to any usage rules applicable to the other Google products. Rental Content. When you rent content, you may view such item of content an unlimited number of times during the period of time displayed at the time of your payment ("Viewing Period") and noted in your confirmation email. Pausing, stopping, or rewinding an item of rented content will not extend your applicable Viewing Period. Each item of rented content may have a different Viewing Period, so please review the Viewing Period before you complete your transaction. Rented content may only be viewed on one device at a time (either online or on an authorized offline device). Purchased Content. When you purchase an item of content, your content will be stored in a digital locker and you may view it an unlimited number of times for during your Locker Period. The “Locker Period” will be for at least 5 years from the date of your purchase (subject to the restrictions described in the YouTube Paid Service Terms of Service). Each item of purchased content may have a different Locker Period and you agree to the Locker Period before you order it. Pausing, stopping, or rewinding purchased content will not extend the Locker Period.  As noted in the YouTube Paid Service Terms of Service, if an item of purchased content becomes unavailable during the five year period from the purchase date, you may request a refund. For purchased content: you may view one stream of each item at a time, you may view up to 3 streams of different items at a time, you may authorize up to 5 devices for offline playback of Locker Video Content at a time and to authorize additional devices, you must deauthorize one of those 5 devices, you may only authorize the same device three times in any 12 month period and de-authorize the same device twice in any 12 month period, you may only deauthorize a total of 2 devices for offline playback every 90 days, and you may only authorize 3 Google accounts on the same device. Stream and offline playback limitations for purchased content apply regardless of which Google product (e.g., Google Play Movies & TV or YouTube) you access the content from"
# print(wrapper.fill(bert_abstract))
question = "Can I get a refund?"

answer_question(question, bert_abstract)


Query has 323 tokens.

Answer: "If an item of purchased content becomes unavailable during the five year period from the purchase date."


# **WEB SCRAPING**

In [86]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

html = urllib.request.urlopen('https://www.youtube.com/t/usage_paycontent').read()
print(text_from_html(html))

Terms of Service Paid Service Terms of Service Paid Service Usage Rules Collecting Society Notices Copyright Notices Community Guidelines YouTube Paid Service Usage Rules  As described in the YouTube Paid Service Terms of Service , how you access or use different types of Paid Services on YouTube may vary. The usage rules applicable to each type of Paid Service are detailed below, and may change over time as we add new features, devices and content to our service. You may also visit the YouTube Help Center for more information.  Movies and Shows Rentals and Purchases   General. Certain types of Paid Services allow you to purchase or rent individual Movies or TV shows made available by third parties. You may view rentals or purchases when online, with a broadband connection and logged onto your YouTube or Google account. You may also view certain rentals and purchases offline by downloading and viewing from a previously authorized device. Authorizing and deauthorizing a device requires 

In [87]:
bert_abstract = text_from_html(html)
question = "Can I get a refund?"

answer_question(question, bert_abstract)

Query has 728 tokens.



RuntimeError: ignored